**Loading Data and creating benchmark model**

In [ ]:
# Defining the path to the Github repository
  file_url = '../Datasets/bank-full.csv'

In [ ]:
# Loading data using pandas
import pandas as pd
bankData = pd.read_csv(file_url,sep=";")
bankData.head()

In [ ]:
# Removing the target variable
Y = bankData.pop('y')

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(bankData, Y, test_size=0.3, random_state=123)

In [ ]:
# Using pipeline to transform categorical variable and numeric variables
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])



In [ ]:
# Defining data types for numeric and categorical features
numeric_features = bankData.select_dtypes(include=['int64', 'float64']).columns
categorical_features = bankData.select_dtypes(include=['object']).columns



In [ ]:
# Defining preprocessor
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
# Defining the estimator for processing and classification
from sklearn.linear_model import LogisticRegression
estimator = Pipeline(steps=[('preprocessor', preprocessor),                      
                           ('classifier',LogisticRegression(random_state=123))])


In [ ]:
# Fit the estimator on the training set
estimator.fit(X_train, y_train)  
print("model score: %.2f" % estimator.score(X_test, y_test)) 

In [ ]:
# Predict on the test set
pred = estimator.predict(X_test)

In [ ]:
# Generating classification report
from sklearn.metrics import classification_report

print(classification_report(pred,y_test))

**Establishing entities and relationship**

In [ ]:
# Creating the Ids for Demographic Entity
bankData['custID'] = bankData.index.values

bankData['custID'] = 'cust' + bankData['custID'].astype(str)

In [ ]:
# Creating AssetId
bankData['AssetId'] = 0
bankData.loc[bankData.housing == 'yes','AssetId']= 1

In [ ]:
# Creating LoanId
bankData['LoanId'] = 0
bankData.loc[bankData.loan == 'yes','LoanId']= 1

In [ ]:
# Creating Financial behaviour ID
bankData['FinbehId'] = 0
bankData.loc[bankData.default == 'yes','FinbehId']= 1

In [ ]:
# Importing necessary libraries
import featuretools as ft
import numpy as np

In [ ]:
# creating the entity set 'Bankentities'
Bankentities = ft.EntitySet(id = 'Bank')

In [ ]:
# Mapping a dataframe to the entityset to form the parent entity
Bankentities.entity_from_dataframe(entity_id = 'Demographic Data', dataframe = bankData, index = 'custID')

In [ ]:
# Mapping to parent entity and setting the relationship
Bankentities.normalize_entity(base_entity_id='Demographic Data', new_entity_id='Assets', index = 'AssetId', 
additional_variables = ['housing'])

Bankentities.normalize_entity(base_entity_id='Demographic Data', new_entity_id='Liability', index = 'LoanId', 
additional_variables = ['loan'])

Bankentities.normalize_entity(base_entity_id='Demographic Data', new_entity_id='FinBehaviour', index = 'FinbehId', 
additional_variables = ['default'])


**Feature Engineering**

In [ ]:
# Creating aggregation and transformation primitives
aggPrimitives=[
        'std', 'min', 'max', 'mean', 
         'last', 'count'
        
]
tranPrimitives=[
        'percentile', 
         'subtract', 'divide']

In [ ]:
# Defining the new set of features
feature_set, feature_names = ft.dfs(entityset=Bankentities, 
target_entity = 'Demographic Data',
agg_primitives=aggPrimitives,
trans_primitives=tranPrimitives, 
max_depth = 2, 
verbose = 1, 
n_jobs = 1)

In [ ]:
# Reindexing the feature_set
feature_set = feature_set.reindex(index=bankData['custID'])
feature_set = feature_set.reset_index()

In [ ]:
# Displaying the feature set 
feature_set.shape

**Cleaning na values and infinity values**

In [ ]:
# Dropping all Ids
X = feature_set[feature_set.columns[~feature_set.columns.str.contains(
    'custID|AssetId|LoanId|FinbehId')]]


In [ ]:
# Replacing all columns with infinity with nan
X = X.replace([np.inf, -np.inf], np.nan)



In [ ]:
# Dropping all columns with nan
X = X.dropna(axis=1, how='any')
X.shape

**Modelling phase**

In [ ]:
# Splitting train and test sets
from sklearn.model_selection import train_test_split

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=123)

In [ ]:
# Creating the preprocessing pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
# Creating the estimator function and fitting the training set
estimator = Pipeline(steps=[('preprocessor', preprocessor),                      
                           ('classifier',LogisticRegression(random_state=123))])
estimator.fit(X_train, y_train)  
print("model score: %.2f" % estimator.score(X_test, y_test)) 

In [ ]:
# Predicting on the test set
pred = estimator.predict(X_test)

In [ ]:
# Generating the classification report
from sklearn.metrics import classification_report

print(classification_report(pred,y_test))